We import the ICD codes and the dataset-specific codes, then encode them into embeddings using a transformer model. To visualize potential clusters to aid cleaning, we apply UMAP for dimensionality reduction and HDBSCAN for cluster detection.